This files creates the training and test datasets for all other notebooks and scripts to reuse. This is the file I would typically use for an exploratory analysis.

In [11]:
import os
import pandas as pd
from helpsk.utility import Timer
from helpsk.pandas import numeric_summary, non_numeric_summary

import plotly.express as px


# Data

In [5]:
import pandas as pd

df = pd.read_csv('/code/daily_restaurant_sales.csv')
df.head()

,rest_id,day,month,weekday,weekend,is_holiday,is_dec,is_nov,competitors_price,discounts,sales
0,0,2016-01-01,1,4,False,True,False,False,2.88,0,79.0
1,0,2016-01-02,1,5,True,False,False,False,2.64,0,57.0
2,0,2016-01-03,1,6,True,False,False,False,2.08,5,294.0
3,0,2016-01-04,1,0,False,False,False,False,3.37,15,676.5
4,0,2016-01-05,1,1,False,False,False,False,3.79,0,66.0


In [7]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
    # Change column type to datetime64[ns] for column: 'day'
    df = df.astype({'day': 'datetime64[ns]'})
    return df

df_clean = clean_data(df.copy())
df_clean.head()

,rest_id,day,month,weekday,weekend,is_holiday,is_dec,is_nov,competitors_price,discounts,sales
0,0,2016-01-01,1,4,False,True,False,False,2.88,0,79.0
1,0,2016-01-02,1,5,True,False,False,False,2.64,0,57.0
2,0,2016-01-03,1,6,True,False,False,False,2.08,5,294.0
3,0,2016-01-04,1,0,False,False,False,False,3.37,15,676.5
4,0,2016-01-05,1,1,False,False,False,False,3.79,0,66.0


In [11]:
from datetime import date, datetime

# convert string to date
date_string = "2021-08-01 00:00:00"
date_object = date.fromisoformat(date_string)
print(type(date_object))
print(date_object)


ValueError: Invalid isoformat string: '2021-08-01 00:00:00'

In [7]:
datetime(2021, 1, 1, 0, 0).date()

datetime.date(2021, 1, 1)

In [8]:
datetime.strptime(min_value, '%Y-%m-%d')

NameError: name 'min_value' is not defined

In [19]:
df_clean.columns.tolist()

['rest_id',
 'day',
 'month',
 'weekday',
 'weekend',
 'is_holiday',
 'is_dec',
 'is_nov',
 'competitors_price',
 'discounts',
 'sales']

In [23]:

import numpy as np
[x for x in np.arange(0.1, 1.1, 0.1)]

[0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0]

In [16]:
import plotly.io as pio
pio.renderers.default = "vscode"

In [17]:
px.histogram(df_clean, x='sales', nbins=30, height=700)

---